加载环境变量

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [2]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

 4. 创建向量存储

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['114ebb87-fff1-41d6-8239-8de09811be40',
 '75150a08-6a1b-483d-af0b-3d1b5ba39c84',
 '51d05eb2-919c-41d4-88d1-6acc40c5f17a',
 'a3c2f0de-9928-4c1d-aea4-9a0c701c73e2',
 '363098a0-a9dc-4a68-a8a0-fedc1a558b89',
 '26512769-3b8e-4645-bfa8-0865e3077946',
 '9a461f86-ae2a-44c4-a744-abe717964488',
 'fc72af50-e5dd-4bed-8d8f-fb217a975fa9',
 'e724d376-bde2-4c1f-8001-e003dd3216d2',
 '8fde3e5f-6400-4aea-89f7-42cf154badcf',
 'c486669e-b267-447a-8ebc-ce4198c8c463',
 'babe671d-8e38-4288-a3c5-2bded6bfe4e9',
 '8585e721-9318-4905-a4dd-2c387213bb2a',
 '35114dd4-1b3b-40dd-84c0-29e12ebac69d',
 'a574a31e-4587-4c9c-a100-9654ee23370c',
 '1d617cb3-9310-429b-b00f-b27bd6444420',
 '4dfdaec6-547a-470c-9c22-e330637063e0',
 '5b0d3a94-53e1-498c-a2e6-43e351bc6165',
 '180234e9-9a8a-4a97-833c-deb5c8775088',
 'e0a4118d-0614-48db-881a-2b4defeff83f',
 'f9818f69-3bf1-4074-8fee-1046c6f05757',
 'a0019ae7-2a7a-462d-b052-8950e393eca3',
 'b6858a25-25d0-4ec4-ab96-5dead8d4ab58',
 '93c16c81-0263-44f7-a8c0-1691304d1634',
 '34a68713-e821-

5. 构建用户查询

In [6]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [7]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用ollama本地大语言模型生成答案

In [9]:
from langchain_ollama import ChatOllama # pip install langchain-ollama
llm = ChatOllama(
    model="qwen2.5:3b",  # 可以根据需要更换其他模型，如 llama2, mistral 等
    request_timeout=300.0  # 增加超时时间
)

9. 生成答案

In [10]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
--------------------------------------------------
回答: 《黑神话：悟空》中的游戏场景包括多个中国古建地标，如大足石刻、小西天、南禅寺、铁佛寺、广胜寺和鹳雀楼等。这些地点在游戏中真实呈现，并为玩家提供了丰富的探索环境。此外，游戏还通过二维和三维动画过场，展现了叙事和主题元素，增强了沉浸感。
